<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2 - Ames Housing Data and Kaggle Challenge

# Executive Summary 

An in-house data science team has been set up within a property agent organisation to predict the prices of future house listings. This information would be used for internal reference only and assist the realtors to determine the valuation and potential of resale of a house from a seller. The original given dataset consists of 82 columns describing the conditions and qualities of the houses. The exploratory data analysis was split between the numeric and categoric features. Our initial findings showed that there were many numeric features which had multicollinearity with one another. By utilising the correlation between the features in comparison with correlation of features and sale price, we managed to reduce the number of numeric features with multicollinearity issue. 

Subsequently, feature engineering was done to the numeric features as well. A lasso model gave the best R-squared and RMSE score. The final features selected consist of only 14 - a significant reduction from the 82 initial columns. This would streamline the potential features that our fellow colleagues, the realtors, would have to look out for in a potential resale of a house. 

## Problem Statement

The organisation is interested to streamline and extract which features from the given Ames Housing Dataset that may be useful in providing an accurate and efficient service for the realtors to predict the future sale prices of houses in Ames, Iowa. 

This would assist the realtors in the process of identifying potential houses of particular resale value and the key features to look out for. 

## Contents: 
Part 1: Data Import and Cleaning

-[Train Data Set](#Train)

-[Test Data Set](#Test)

-[Summary](#Summary)

Part 2: Exploratory Data Analysis and Statistics

Part 3: Modelling

## Data Sets

* [`train.csv`](./datasets/train.csv): Ames Housing Dataset
* [`train_clean.csv`](./datasets/train_clean.csv): Ames Housing Dataset after cleaning
* [`train_selected.csv`](./datasets/train_selected.csv): Ames Housing Dataset features selected
* [`test.csv`](./datasets/test.csv): Ames Housing Dataset
* [`test_clean.csv`](./datasets/test_clean.csv): Ames Housing Dataset after cleaning
* [`test_selected.csv`](./datasets/test_selected.csv): Ames Housing Dataset features selected
* [`submission_lasso.csv`](./datasets/submission_lasso.csv): Final export for kaggle submission

There are two sets of given data, train.csv and test.csv. The dataset consists of 82 variables that may be useful in predicting sale price. Our objective is to streamline and select the appropriate variables to be used as features to predict the prices of the houses in Ames. 

# Part 1

## Data Import and Cleaning

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score,train_test_split

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

<a id="Train"></a>
## Train
Import train.csv data set

In [2]:
train = pd.read_csv('./datasets/train.csv')

In [3]:
print(train.shape)
train.head()

(2051, 81)


,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

In [5]:
train.isnull().sum().sort_values(ascending=False)

Pool QC         2042
Misc Feature    1986
Alley           1911
Fence           1651
Fireplace Qu    1000
                ... 
Heating QC         0
Central Air        0
Electrical         0
1st Flr SF         0
SalePrice          0
Length: 81, dtype: int64

In [7]:
# total number of null values in each feature that has null values
(pd.isnull(train).sum()[pd.isnull(train).sum() > 0]).sort_values(ascending=False)

Pool QC           2042
Misc Feature      1986
Alley             1911
Fence             1651
Fireplace Qu      1000
Lot Frontage       330
Garage Yr Blt      114
Garage Cond        114
Garage Qual        114
Garage Finish      114
Garage Type        113
Bsmt Exposure       58
BsmtFin Type 2      56
Bsmt Cond           55
Bsmt Qual           55
BsmtFin Type 1      55
Mas Vnr Area        22
Mas Vnr Type        22
Bsmt Half Bath       2
Bsmt Full Bath       2
Total Bsmt SF        1
Bsmt Unf SF          1
BsmtFin SF 2         1
Garage Cars          1
Garage Area          1
BsmtFin SF 1         1
dtype: int64

**Findings**

After considering the features and data types, concluded that NaN in numerical columns should be replaced with 0 and NaN in categorical columns should be replaced with 'none' . 

The null values usually represent a missing feature. 

Not possible to replace NaN with mean, mode or median, eg if there is no basement then there is no basement, cannot take the average. 

In [7]:
train.duplicated().sum()

0

There are no duplicates

In [8]:
list_nan_cat = ['Pool QC', 'Misc Feature', 'Alley', 'Fence', 'Fireplace Qu', 'Garage Cond', 'Garage Qual', 'Garage Finish', 'Garage Type', 'Bsmt Exposure', 'BsmtFin Type 2', 'Bsmt Cond', 'Bsmt Qual', 'BsmtFin Type 1', 'Mas Vnr Type']

In [9]:
list_nan_num = ['Lot Frontage', 'Garage Yr Blt', 'Mas Vnr Area', 'Bsmt Half Bath', 'Bsmt Full Bath', 'Total Bsmt SF', 'Bsmt Unf SF', 'BsmtFin SF 2', 'Garage Cars', 'Garage Area', 'BsmtFin SF 1']

In [11]:
# fill categoric features null with 'none'
train[list_nan_cat] = train[list_nan_cat].fillna('none')

In [12]:
# fill numeric features null with 0
train[list_nan_num] = train[list_nan_num].fillna(0)

In [13]:
# double check
train.isnull().sum().sort_values(ascending=False)

Id              0
Heating QC      0
Garage Type     0
Fireplace Qu    0
Fireplaces      0
               ..
Mas Vnr Type    0
Exterior 2nd    0
Exterior 1st    0
Roof Matl       0
SalePrice       0
Length: 81, dtype: int64

In [18]:
print(train.shape)
train.head()

(2051, 81)


,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,0.0,13517,Pave,none,IR1,Lvl,...,0,0,none,none,none,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,none,IR1,Lvl,...,0,0,none,none,none,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,none,Reg,Lvl,...,0,0,none,none,none,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,none,Reg,Lvl,...,0,0,none,none,none,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,none,IR1,Lvl,...,0,0,none,none,none,0,3,2010,WD,138500


<a id="Test"></a>
## Test
Repeat similar steps for test.csv

In [19]:
test = pd.read_csv('./datasets/test.csv')

In [20]:
print(test.shape)
test.head()

(878, 80)


,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [21]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               878 non-null    int64  
 1   PID              878 non-null    int64  
 2   MS SubClass      878 non-null    int64  
 3   MS Zoning        878 non-null    object 
 4   Lot Frontage     718 non-null    float64
 5   Lot Area         878 non-null    int64  
 6   Street           878 non-null    object 
 7   Alley            58 non-null     object 
 8   Lot Shape        878 non-null    object 
 9   Land Contour     878 non-null    object 
 10  Utilities        878 non-null    object 
 11  Lot Config       878 non-null    object 
 12  Land Slope       878 non-null    object 
 13  Neighborhood     878 non-null    object 
 14  Condition 1      878 non-null    object 
 15  Condition 2      878 non-null    object 
 16  Bldg Type        878 non-null    object 
 17  House Style     

In [22]:
(pd.isnull(test).sum()[pd.isnull(test).sum() > 0]).sort_values(ascending=False)

Pool QC           874
Misc Feature      837
Alley             820
Fence             706
Fireplace Qu      422
Lot Frontage      160
Garage Yr Blt      45
Garage Finish      45
Garage Qual        45
Garage Cond        45
Garage Type        44
Bsmt Exposure      25
BsmtFin Type 1     25
BsmtFin Type 2     25
Bsmt Cond          25
Bsmt Qual          25
Mas Vnr Area        1
Mas Vnr Type        1
Electrical          1
dtype: int64

In [23]:
test.duplicated().sum()

0

In [24]:
test[list_nan_cat] = test[list_nan_cat].fillna('none')

In [25]:
test[list_nan_num] = test[list_nan_num].fillna(0)

In [26]:
train.isnull().sum().sort_values(ascending=False)

Id              0
Heating QC      0
Garage Type     0
Fireplace Qu    0
Fireplaces      0
               ..
Mas Vnr Type    0
Exterior 2nd    0
Exterior 1st    0
Roof Matl       0
SalePrice       0
Length: 81, dtype: int64

In [27]:
print(test.shape)
test.head()

(878, 80)


,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,none,none,none,0,4,2006,WD
1,2718,905108090,90,RL,0.0,9662,Pave,none,IR1,Lvl,...,0,0,0,none,none,none,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,none,IR1,Lvl,...,0,0,0,none,none,none,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,none,Reg,Lvl,...,0,0,0,none,none,none,0,7,2007,WD
4,625,535105100,20,RL,0.0,9500,Pave,none,IR1,Lvl,...,0,185,0,none,none,none,0,7,2009,WD


In [32]:
train.columns = train.columns.str.lower()

In [36]:
train.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

In [38]:
test.columns = test.columns.str.lower()

In [39]:
test.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

In [43]:
# export to csv
train.to_csv('./datasets/train_clean.csv', index=False)

In [44]:
# export to csv
test.to_csv('./datasets/test_clean.csv', index=False)

<a id="Summary"></a>
## Summary

There are no duplicate rows in the datasets provided. With reference to the data dictionary, it became clear that there are many null values that are not actually missing values but instead represent that there is no such feature in the house. For example, if the house has no garage then null values appear multiple times across the variables related to garage for the particular house. 

As such, a value of 0 was impute to the numeric variables while a string 'none' was impute to the categoric variables. 